In [66]:
import os
import re
import pandas as pd
import numpy as np

In [67]:
df = pd.read_parquet('../../acts_parquet/lic_435_acts_200x_2018_2020_iob.parquet')
df

,ORGAO_LICITANTE,NUM_LICITACAO,OBJ_LICITACAO,MODALIDADE_LICITACAO,PROCESSO,DATA_ABERTURA,VALOR_ESTIMADO,CODIGO_SISTEMA_COMPRAS,SISTEMA_COMPRAS,NOME_RESPONSAVEL,treated_text,IOB
0,demais orgaos participantes do Sistema de Regi...,001/2020,Registro de Precos visando a eventual aquisica...,PREGAO ELETRONICO,00040-00017799/2019-42,08/01/2020,"170.704,26",None,Comprasnet,KARLA REGINA DA SILVA ROCHA,AVISO DE LICITACAO PREGAO ELETRONICO No 001/20...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,METRO-DF,03/2019,contratacao de empresa especializada para o fo...,PREGAO ELETRONICO,00097-00003659/2018-32,None,sigiloso,925046,www.comprasgovernamentais.gov.br,ANDERSON LUIZ SENNA COSTA,PREGAO ELETRONICO No 03/2019 UASG UASG 925046 ...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
2,Secretaria de Estado de Seguranca Publica do D...,31/2019,Registro de Precos para futura aquisicao de eq...,PREGAO ELETRONICO,00050-00034379/2019-93,03/01/2020,None,450107,www.comprasgovernamentais.gov.br,AMILCAR UBIRATAN URACH VIEIRA,AVISO DE ABERTURA DE LICITACAO SRP SRP PREGAO ...,O O O O O O O B-MODALIDADE_LICITACAO I-MODALID...
3,None,301/2019,Registro de precos para aquisicao de EPIEquipa...,PREGAO ELETRONICO,092.006186/2019,16/01/2020,None,974200,www.comprasnet.gov.br,NARA MAGALHAES DA SILVA,AVISOS DE LICITACAO PREGAO ELETRONICO PE No 30...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
4,None,116/2018,Registro de Precos visando a eventual aquisica...,PREGAO ELETRONICO,00410-00024534/2017-65,20/11/2018,"767.907,37",None,www.compras.df.gov.br,GERARDA DA SILVA CARVALHO,AVISOS DE LICITACOES PREGAO ELETRONICO No 116/...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
...,...,...,...,...,...,...,...,...,...,...,...,...
430,None,442/2004,Aquisição de material de consumo farmacológico...,PREGÃO,None,24/08/2004,None,None,None,None,PREGÃO Nº 442/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
431,None,451/2004,"Aquisição de medicamentos (Lamivudina, octreot...",PREGÃO,None,02/09/2004,None,None,None,None,PREGÃO Nº 451/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
432,None,024/2002,aquisi- ção ção de conectores,Tomada de Preços de Material,None,10/02/2003,None,None,None,None,AVISO DE LICITAÇÃO A COMPANHIA ENERGÉTICA DE B...,O O O O O O O O O O O O O O O O O O O O O O O ...
433,None,005/2002,Aquisição de materiais permanentes para uso na...,TOMADA DE PREÇOS,053.000.918/2002,11/10/2002,None,None,None,None,AVISO DE LICITAÇÃO PROCESSO N.º 053.000.918/20...,O O O O O O B-PROCESSO I-PROCESSO O O B-MODALI...


In [68]:
df.isnull().sum()

ORGAO_LICITANTE           274
NUM_LICITACAO               7
OBJ_LICITACAO               2
MODALIDADE_LICITACAO        8
PROCESSO                   67
DATA_ABERTURA              38
VALOR_ESTIMADO            217
CODIGO_SISTEMA_COMPRAS    251
SISTEMA_COMPRAS            89
NOME_RESPONSAVEL          242
treated_text                0
IOB                         0
dtype: int64

In [69]:
ORGAO_LICITANTE = []
VALOR_ESTIMADO = []
CODIGO_SISTEMA_COMPRAS = []
NOME_RESPONSAVEL = []


for i in range(len(df)):
    if df['ORGAO_LICITANTE'][i] != None:
        ORGAO_LICITANTE.append(i)
    if df['VALOR_ESTIMADO'][i] != None:
        VALOR_ESTIMADO.append(i)
    if df['CODIGO_SISTEMA_COMPRAS'][i] != None:
        CODIGO_SISTEMA_COMPRAS.append(i)
    if df['NOME_RESPONSAVEL'][i] != None:
        NOME_RESPONSAVEL.append(i)

In [70]:
len(ORGAO_LICITANTE), len(VALOR_ESTIMADO), len(CODIGO_SISTEMA_COMPRAS), len(NOME_RESPONSAVEL)

(161, 218, 184, 193)

In [71]:
ORGAO_LICITANTE_1 = ORGAO_LICITANTE[:(len(ORGAO_LICITANTE)*70)//100]
ORGAO_LICITANTE_2 = ORGAO_LICITANTE[(len(ORGAO_LICITANTE)*70)//100:]

VALOR_ESTIMADO_set = set(VALOR_ESTIMADO).difference(set(ORGAO_LICITANTE_2))
VALOR_ESTIMADO_1 = list(VALOR_ESTIMADO_set)[:(len(VALOR_ESTIMADO_set)*70)//100]
VALOR_ESTIMADO_2 = set(list(VALOR_ESTIMADO_set)[(len(VALOR_ESTIMADO_set)*70)//100:]).difference(set(ORGAO_LICITANTE_1))

CODIGO_SISTEMA_COMPRAS_set = set(CODIGO_SISTEMA_COMPRAS).difference(set(ORGAO_LICITANTE_2))
CODIGO_SISTEMA_COMPRAS_set = set(CODIGO_SISTEMA_COMPRAS_set).difference(set(VALOR_ESTIMADO_2))
CODIGO_SISTEMA_COMPRAS_1 = list(CODIGO_SISTEMA_COMPRAS_set)[:(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100]
CODIGO_SISTEMA_COMPRAS_2 = set(list(CODIGO_SISTEMA_COMPRAS_set)[(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100:]).difference(set(ORGAO_LICITANTE_1))
CODIGO_SISTEMA_COMPRAS_2 = set(list(CODIGO_SISTEMA_COMPRAS_set)[(len(CODIGO_SISTEMA_COMPRAS_set)*70)//100:]).difference(set(VALOR_ESTIMADO_1))

NOME_RESPONSAVEL_set = set(NOME_RESPONSAVEL).difference(set(ORGAO_LICITANTE_2))
NOME_RESPONSAVEL_set = set(NOME_RESPONSAVEL_set).difference(set(VALOR_ESTIMADO_2))
NOME_RESPONSAVEL_set = set(NOME_RESPONSAVEL_set).difference(set(CODIGO_SISTEMA_COMPRAS_2))
NOME_RESPONSAVEL_1 = list(NOME_RESPONSAVEL_set)[:(len(NOME_RESPONSAVEL_set)*70)//100]
NOME_RESPONSAVEL_2 = set(list(NOME_RESPONSAVEL_set)[(len(NOME_RESPONSAVEL_set)*70)//100:]).difference(set(ORGAO_LICITANTE_1))
NOME_RESPONSAVEL_2 = set(list(NOME_RESPONSAVEL_set)[(len(NOME_RESPONSAVEL_set)*70)//100:]).difference(set(VALOR_ESTIMADO_1))
NOME_RESPONSAVEL_2 = set(list(NOME_RESPONSAVEL_set)[(len(NOME_RESPONSAVEL_set)*70)//100:]).difference(set(CODIGO_SISTEMA_COMPRAS_1))

In [72]:
len(ORGAO_LICITANTE_1), len(ORGAO_LICITANTE_2), len(VALOR_ESTIMADO_1), len(VALOR_ESTIMADO_2), len(CODIGO_SISTEMA_COMPRAS_1), len(CODIGO_SISTEMA_COMPRAS_2), len(NOME_RESPONSAVEL_1), len(NOME_RESPONSAVEL_2)

(112, 49, 136, 58, 101, 26, 118, 32)

In [73]:
lic = []

lic.extend(ORGAO_LICITANTE_1)
lic.extend(VALOR_ESTIMADO_1)
lic.extend(CODIGO_SISTEMA_COMPRAS_1)
lic.extend(NOME_RESPONSAVEL_1)

lic = sorted(list(set(lic)))

In [74]:
not_lic = []

not_lic.extend(ORGAO_LICITANTE_2)
not_lic.extend(VALOR_ESTIMADO_2)
not_lic.extend(CODIGO_SISTEMA_COMPRAS_2)
not_lic.extend(NOME_RESPONSAVEL_2)

not_lic = sorted(list(set(not_lic)))

In [75]:
len(lic), len(not_lic)

(179, 165)

In [76]:
for i in lic:
    for j in not_lic:
        if i == j:
            not_lic.pop(not_lic.index(j))

for i in lic:
    for j in not_lic:
        if i == j:
            print(i)

In [77]:
df_lic = df.iloc[lic]
df_not_lic = df.iloc[not_lic]

In [78]:
len(df_lic), len(df_not_lic)

(179, 129)

In [79]:
df_lic.isnull().sum()

ORGAO_LICITANTE           67
NUM_LICITACAO              5
OBJ_LICITACAO              1
MODALIDADE_LICITACAO       7
PROCESSO                   5
DATA_ABERTURA              9
VALOR_ESTIMADO            42
CODIGO_SISTEMA_COMPRAS    55
SISTEMA_COMPRAS           20
NOME_RESPONSAVEL           6
treated_text               0
IOB                        0
dtype: int64

In [80]:
df_not_lic.isnull().sum()

ORGAO_LICITANTE            80
NUM_LICITACAO               2
OBJ_LICITACAO               1
MODALIDADE_LICITACAO        1
PROCESSO                   14
DATA_ABERTURA              19
VALOR_ESTIMADO             48
CODIGO_SISTEMA_COMPRAS     69
SISTEMA_COMPRAS            29
NOME_RESPONSAVEL          109
treated_text                0
IOB                         0
dtype: int64

In [81]:
df_real = df.drop(df_lic.index)
df_real = df_real.drop(df_not_lic.index)

In [82]:
len(df_real)

127

In [83]:
train = pd.concat([df_lic, df_real[:int(len(df)*0.7) - len(df_lic)]])
test = pd.concat([df_real[int(len(df)*0.7) - len(df_lic):], df_not_lic])

In [84]:
len(train), len(test), len(df)

(304, 131, 435)

In [85]:
test.isnull().sum()

ORGAO_LICITANTE            82
NUM_LICITACAO               2
OBJ_LICITACAO               1
MODALIDADE_LICITACAO        1
PROCESSO                   15
DATA_ABERTURA              19
VALOR_ESTIMADO             50
CODIGO_SISTEMA_COMPRAS     71
SISTEMA_COMPRAS            31
NOME_RESPONSAVEL          111
treated_text                0
IOB                         0
dtype: int64

In [86]:
train.isnull().sum()

ORGAO_LICITANTE           192
NUM_LICITACAO               5
OBJ_LICITACAO               1
MODALIDADE_LICITACAO        7
PROCESSO                   52
DATA_ABERTURA              19
VALOR_ESTIMADO            167
CODIGO_SISTEMA_COMPRAS    180
SISTEMA_COMPRAS            58
NOME_RESPONSAVEL          131
treated_text                0
IOB                         0
dtype: int64

In [87]:
i = 0
for col in list(df.columns):
    print(f'{col}: {((len(train) - list(train.isnull().sum())[i]) * 100) / (len(df) - list(df.isnull().sum())[i])}%')
    i += 1

ORGAO_LICITANTE: 69.56521739130434%
NUM_LICITACAO: 69.85981308411215%
OBJ_LICITACAO: 69.9769053117783%
MODALIDADE_LICITACAO: 69.55503512880563%
PROCESSO: 68.47826086956522%
DATA_ABERTURA: 71.78841309823677%
VALOR_ESTIMADO: 62.84403669724771%
CODIGO_SISTEMA_COMPRAS: 67.3913043478261%
SISTEMA_COMPRAS: 71.09826589595376%
NOME_RESPONSAVEL: 89.63730569948187%
treated_text: 69.88505747126437%
IOB: 69.88505747126437%


In [88]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [89]:
train.to_parquet('train.parquet')
test.to_parquet('test.parquet')

In [92]:
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

len(train), len(test), len(train) + len(test)

(304, 131, 435)